In [191]:
from jsonobject import *
import numpy as np 
import random
import math
import matplotlib as mpl
import matplotlib.pyplot as plt

In [192]:
random.seed(2021)

# Classes

In [193]:
# Convolution
class Convolution(JsonObject):
    kernel = IntegerProperty()
    padding = StringProperty()
    stride = IntegerProperty()
    nb_filter = IntegerProperty()
    fct_activation = StringProperty()

In [194]:
# Input 
class InputLayer(JsonObject):
    shape = ListProperty(int)

In [195]:
# Pooling Avg/Max
class Pooling(JsonObject):
    op = StringProperty()
    kernel = IntegerProperty(default=2)
    padding = StringProperty(default="valid")
    stride = IntegerProperty(default=None)

In [196]:
# Class Flatten
class Flatten(JsonObject):
    pass

In [197]:
# Dense --> Fully connected layer
class Dense(JsonObject):
    nb_neurones =  IntegerProperty()
    fct_activation = StringProperty()

In [198]:
# Batchnorm
class BatchNormalisation(JsonObject):
    epsilon = FloatProperty()

In [199]:
# Global pooling 
class GlobalPooling(JsonObject):
    op = StringProperty()

In [200]:
# classe for the different possible merge add/concatenate
class Merge(JsonObject):
    m_type = StringProperty()
    sub_SM_1 = ListProperty(JsonObject)
    sub_SM_2 = ListProperty(JsonObject)
    
    def equals_sm1_sm2():
        for i,j in sub_SM_1,sub_SM_2:
            if (i != j):
                return False
        return True

In [201]:
# class dropout
class Dropout(JsonObject):
    dropout_rate = FloatProperty()   

In [202]:
# class that represente when the way of merge is empty
class Empty(JsonObject):
    pass

# Value

In [203]:
# Table of hyperparameter value
kernel_value = [1, 2, 3, 4, 5, 6, 7]
stride_value = [1, 2, 3, 4, 5, 6, 7]
padding_value = ["valid", "same"]
fct_activation_value = ["tanh", "relu", "selu"]

epsilon_value = [0, 1.001e-5, 0.001, 1.1e-5, 1.1e-7]
dropout_value = [0, .1, .4, .5, .8, .01, .001]

nb_layer_value = [1,2,3]
nb_block_densenet_value = [1,2,3,4]

nb_class = 10 # nb_class
op_value = ['avg',"max"]

m_type_value = ["add", "concatenate"]

classification = [Flatten, GlobalPooling]

# Functions

In [204]:
def stop(size_archi, x):
    prob = (math.log(2*size_archi)*x)-(2*x)
    if ( prob < random.randrange(101)):
        return True
    else : return False

In [205]:
def create_json_file(archi, file_name):
    directory = 'architecture_json/'
    
    # reset file
    archi_file = open(directory+file_name, "w")
    archi_file.close()
    
    # create file
    
    # open file
    archi_file = open(directory+file_name, "a")
    
    archi_file.write("""[
    """)
    
    archi_size = len(archi)
    i = 0
    for l in archi:

        str_layer = """\t{
            'class':'"""
        str_layer += l.__class__.__name__
        str_layer +="""',\n\t\t\t'parameters':"""
        str_layer += str(l.to_json())
        str_layer += """\n\t\t}"""
        if(i < archi_size-1):
            str_layer += ""","""
            i+=1
        str_layer = str_layer.replace("'","\"")
        
        archi_file.write(str_layer)
        
    archi_file.write("""\n]""")
    archi_file.close()

In [206]:
# add layer to architecture 

def addLayer(archi, layer, output_size):
    input_size = output_size
    feature_extra = archi[1:]
    
    if( isinstance(layer, Dropout) | isinstance(layer, BatchNormalisation)):
        archi.append(layer)
        return 1, archi, output_size
    
    # add layer if the architecture is empty
    elif(feature_extra == []):
        archi.append(layer)
        return 1, archi, calcul_output(input_size, layer)
    
    else :
        
        # if we couldn't reduce more
        #if(input_size == 1):
        #    return 0, archi, input_size
        
        # if the output size got more than 1 we can add new layer
        if(output_size >= 1): 
            output_size = calcul_output(input_size, layer)
            
            # if output size got negate is that the layer we want to add is wrong
            if(output_size < 1 ):
                return -1, archi, input_size
            # if output size is bigger than 0 we can add new layer and continue
            elif(output_size > 0):
                archi.append(layer)
                return 1, archi, output_size
            # this should not append
            else:
                return "Somethink wrong"
    # this should not append
    return "Error"

In [211]:
# compute the output 
# valid vs same
def calcul_output(input_size, l):
    output_size = 0
    if(l.padding == "valid"):
        kernel = l.kernel
        stride = l.stride
        while(input_size>=kernel):
            input_size -= stride
            output_size += 1   
    else:
        stride = l.stride
        if(input_size%stride == 0):
            output_size = int(input_size/stride)
        else:
            output_size = int(input_size/stride)+1
    return output_size

In [208]:
def build_merge(archi, output_size):
    sub_sm1 = list()
    sub_sm2 = list()
    m_type = m_type_value[random.randrange(2)]
    
    
    
    
    m = Merge(m_type=m_type,
             sub_SM_1=sub_sm1,
             sub_SM_2=sub_sm2)
    
    
    if(m.equals_sm1_sm2):
        #build_merge(archi, output_size)
        print("equals")
    else: 
        archi.append(m)
        return archi , output_size

In [ ]:
# Convolution, Pooling, BatchNormalisation,  Dropout, Merge, Empty
# Return list of layer compared to last layer for the sub state machine
def getSub_SM_Layer():
    

In [209]:
# Convolution, Pooling, BatchNormalisation, Dropout, Merge
# Return list of layer compared to last layer
def getlayer(last_layer, prev_last_layer):
    list_layer = list()
    if(isinstance(last_layer, InputLayer)):
        list_layer = [Convolution, BatchNormalisation, Merge]
    elif(isinstance(last_layer, Convolution)):
        list_layer = [Convolution, Pooling, BatchNormalisation,  Dropout, Merge]
    elif(isinstance(last_layer, Pooling)):
        list_layer = [Convolution, Merge]
    elif(isinstance(last_layer, BatchNormalisation)):
        if(isinstance(prev_last_layer, Convolution)):
            list_layer = [Convolution, Merge, Pooling, Dropout]
        else: list_layer = [Convolution]
    elif(isinstance(last_layer, Dropout)):
        list_layer = [Pooling]
    return list_layer

# Creation Architecture

In [210]:
architecture = list()

nb_filter_value = 1 # init nb feature map
output_size = 28

# add input Layer
architecture.append(InputLayer(shape=[28,28,1]))

# add extraction feature (succession of Pooling/convolution)
# Pooling can't be follow by a Pooling
pooling = True
code = 1 # we can add new layer
j = 6
last_layer = architecture[len(architecture)-1]

while( ((isinstance(last_layer, Dropout)) | (isinstance(last_layer, BatchNormalisation))) | 
      (output_size == 28) | 
      ((stop(len(architecture),7)) & (code == 1))):
    list_layer = getlayer(architecture[len(architecture)-1], architecture[len(architecture)-2])
    layer = list_layer[random.randrange(len(list_layer))]
    
    if (layer == Merge):
        build_merge(architecture, output_size)
    else:
        
        if(layer == Convolution):
            nb_filter_value = nb_filter_value*j
            j=2
        
        # Kernel <= output/2
        if(output_size == 2):
            kernel_value_filtered = [1,2]
        elif(output_size == 1):
            kernel_value_filtered = [1]
        else: 
            kernel_value_filtered = [value for value in kernel_value if value <= math.ceil(output_size/2)]

        kernel=kernel_value_filtered[random.randrange(len(kernel_value_filtered))]
        
        # kernel >= stride
        stride_value_filtered = [value for value in stride_value if value <= kernel]
        stride = stride_value_filtered[random.randrange(len(stride_value_filtered))]
    
        # if layer is Convolution
        if(layer == Convolution):
            add_layer = Convolution(
                kernel=kernel, 
                padding=padding_value[random.randrange(2)], 
                stride=stride, 
                nb_filter= nb_filter_value,
                fct_activation=fct_activation_value[random.randrange(3)]
            )
        # if is pooling
        elif(layer == Pooling):
            add_layer = Pooling(
                op = op_value[random.randrange(2)],
                kernel=kernel, 
                padding=padding_value[random.randrange(2)], 
                stride=stride
            )
        #if is BN
        elif(layer == BatchNormalisation):
            add_layer = BatchNormalisation(epsilon = epsilon_value[random.randrange(len(epsilon_value))])
        
        #if dropout
        elif(layer == Dropout):
            add_layer = Dropout(dropout_rate = dropout_value[random.randrange(len(dropout_value))])
            
        else: 
            print("error")
        
        code, architecture, output_size = addLayer(architecture, add_layer, output_size)
        print(output_size)
        last_layer = architecture[len(architecture)-1]
        print(str(code) + 'code')


try:
    nb_filter_value = architecture[len(architecture)-1]['nb_filter']
except:
    nb_filter_value = int(nb_filter_value/2)

        
        
        
        
print(architecture)


# create file
file_name = "archi_V4_test.json"
print(file_name)
print("")
print("")
create_json_file(architecture, file_name)


equals
equals
28
1code
hello 28
25
1code
hello 25
5
1code
5
1code
hello 5
3
1code
hello 3
3
1code
hello 3
3
1code
3
1code
equals
equals
3
1code
hello 3
3
1code
hello 3
2
1code
2
1code
hello 2
2
1code
hello 2
2
1code
hello 2
2
1code
hello 2
2
1code
equals
equals
equals
hello 2
2
1code
[InputLayer(shape=[28, 28, 1]), BatchNormalisation(epsilon=0.001), Convolution(fct_activation='tanh', kernel=4, nb_filter=6, padding='valid', stride=1), Convolution(fct_activation='tanh', kernel=6, nb_filter=12, padding='same', stride=6), Dropout(dropout_rate=0.8), Pooling(kernel=3, op='max', padding='same', stride=2), Convolution(fct_activation='relu', kernel=1, nb_filter=24, padding='same', stride=1), Convolution(fct_activation='relu', kernel=1, nb_filter=48, padding='same', stride=1), BatchNormalisation(epsilon=1.1e-07), Dropout(dropout_rate=0.0), Pooling(kernel=1, op='max', padding='valid', stride=1), Convolution(fct_activation='relu', kernel=2, nb_filter=96, padding='same', stride=2), Dropout(dropout_